CER005 - Install new Root CA certificate
========================================

Use this notebook to install a new Root CA certificate to a cluster.

Steps
-----

### Parameters

In [ ]:
local_certificate_dir = "mssql-cluster-root-ca"

ca_certificate_file_name = "cacert.pem"
target_ca_certificate_file_name = "cacert.pem"

test_cert_store_root = "/var/opt/secrets/test-certificates"
timeout = 600  # amount of time to wait before cluster is healthy:  default to 10 minutes
check_interval = 10  # amount of time between health checks - default 10 seconds
min_pod_count = 10  # minimum number of healthy pods required to assert health

### Common functions

Define helper functions used in this notebook.

In [ ]:
# Define `run` function for transient fault handling, suggestions on error, and scrolling updates on Windows
import sys
import os
import re
import platform
import shlex
import shutil
import datetime

from subprocess import Popen, PIPE
from IPython.display import Markdown

retry_hints = {} # Output in stderr known to be transient, therefore automatically retry
error_hints = {} # Output in stderr where a known SOP/TSG exists which will be HINTed for further help
install_hint = {} # The SOP to help install the executable if it cannot be found

def run(cmd, return_output=False, no_output=False, retry_count=0, base64_decode=False, return_as_json=False, regex_mask=None):
    """Run shell command, stream stdout, print stderr and optionally return output

    NOTES:

    1.  Commands that need this kind of ' quoting on Windows e.g.:

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='data-pool')].metadata.name}

        Need to actually pass in as '"':

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='"'data-pool'"')].metadata.name}

        The ' quote approach, although correct when pasting into Windows cmd, will hang at the line:
        
            `iter(p.stdout.readline, b'')`

        The shlex.split call does the right thing for each platform, just use the '"' pattern for a '
    """
    MAX_RETRIES = 5
    output = ""
    retry = False

    # When running `azdata sql query` on Windows, replace any \n in """ strings, with " ", otherwise we see:
    #
    #    ('HY090', '[HY090] [Microsoft][ODBC Driver Manager] Invalid string or buffer length (0) (SQLExecDirectW)')
    #
    if platform.system() == "Windows" and cmd.startswith("azdata sql query"):
        cmd = cmd.replace("\n", " ")

    # shlex.split is required on bash and for Windows paths with spaces
    #
    cmd_actual = shlex.split(cmd)

    # Store this (i.e. kubectl, python etc.) to support binary context aware error_hints and retries
    #
    user_provided_exe_name = cmd_actual[0].lower()

    # When running python, use the python in the ADS sandbox ({sys.executable})
    #
    if cmd.startswith("python "):
        cmd_actual[0] = cmd_actual[0].replace("python", sys.executable)

        # On Mac, when ADS is not launched from terminal, LC_ALL may not be set, which causes pip installs to fail
        # with:
        #
        #    UnicodeDecodeError: 'ascii' codec can't decode byte 0xc5 in position 4969: ordinal not in range(128)
        #
        # Setting it to a default value of "en_US.UTF-8" enables pip install to complete
        #
        if platform.system() == "Darwin" and "LC_ALL" not in os.environ:
            os.environ["LC_ALL"] = "en_US.UTF-8"

    # When running `kubectl`, if AZDATA_OPENSHIFT is set, use `oc`
    #
    if cmd.startswith("kubectl ") and "AZDATA_OPENSHIFT" in os.environ:
        cmd_actual[0] = cmd_actual[0].replace("kubectl", "oc")

    # To aid supportability, determine which binary file will actually be executed on the machine
    #
    which_binary = None

    # Special case for CURL on Windows.  The version of CURL in Windows System32 does not work to
    # get JWT tokens, it returns "(56) Failure when receiving data from the peer".  If another instance
    # of CURL exists on the machine use that one.  (Unfortunately the curl.exe in System32 is almost
    # always the first curl.exe in the path, and it can't be uninstalled from System32, so here we
    # look for the 2nd installation of CURL in the path)
    if platform.system() == "Windows" and cmd.startswith("curl "):
        path = os.getenv('PATH')
        for p in path.split(os.path.pathsep):
            p = os.path.join(p, "curl.exe")
            if os.path.exists(p) and os.access(p, os.X_OK):
                if p.lower().find("system32") == -1:
                    cmd_actual[0] = p
                    which_binary = p
                    break

    # Find the path based location (shutil.which) of the executable that will be run (and display it to aid supportability), this
    # seems to be required for .msi installs of azdata.cmd/az.cmd.  (otherwise Popen returns FileNotFound) 
    #
    # NOTE: Bash needs cmd to be the list of the space separated values hence shlex.split.
    #
    if which_binary == None:
        which_binary = shutil.which(cmd_actual[0])

    # Display an install HINT, so the user can click on a SOP to install the missing binary
    #
    if which_binary == None:
        print(f"The path used to search for '{cmd_actual[0]}' was:")
        print(sys.path)

        if user_provided_exe_name in install_hint and install_hint[user_provided_exe_name] is not None:
            display(Markdown(f'HINT: Use [{install_hint[user_provided_exe_name][0]}]({install_hint[user_provided_exe_name][1]}) to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)")
    else:   
        cmd_actual[0] = which_binary

    start_time = datetime.datetime.now().replace(microsecond=0)

    cmd_display = cmd
    if regex_mask is not None:
        regex = re.compile(regex_mask)
        cmd_display = re.sub(regex, '******', cmd)
        
    print(f"START: {cmd_display} @ {start_time} ({datetime.datetime.utcnow().replace(microsecond=0)} UTC)")
    print(f"       using: {which_binary} ({platform.system()} {platform.release()} on {platform.machine()})")
    print(f"       cwd: {os.getcwd()}")

    # Command-line tools such as CURL and AZDATA HDFS commands output
    # scrolling progress bars, which causes Jupyter to hang forever, to
    # workaround this, use no_output=True
    #

    # Work around a infinite hang when a notebook generates a non-zero return code, break out, and do not wait
    #
    wait = True 

    try:
        if no_output:
            p = Popen(cmd_actual)
        else:
            p = Popen(cmd_actual, stdout=PIPE, stderr=PIPE, bufsize=1)
            with p.stdout:
                for line in iter(p.stdout.readline, b''):
                    line = line.decode()
                    if return_output:
                        output = output + line
                    else:
                        if cmd.startswith("azdata notebook run"): # Hyperlink the .ipynb file
                            regex = re.compile('  "(.*)"\: "(.*)"') 
                            match = regex.match(line)
                            if match:
                                if match.group(1).find("HTML") != -1:
                                    display(Markdown(f' - "{match.group(1)}": "{match.group(2)}"'))
                                else:
                                    display(Markdown(f' - "{match.group(1)}": "[{match.group(2)}]({match.group(2)})"'))

                                    wait = False
                                    break # otherwise infinite hang, have not worked out why yet.
                        else:
                            print(line, end='')

        if wait:
            p.wait()
    except FileNotFoundError as e:
        if install_hint is not None:
            display(Markdown(f'HINT: Use {install_hint} to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)") from e

    exit_code_workaround = 0 # WORKAROUND: azdata hangs on exception from notebook on p.wait()

    if not no_output:
        for line in iter(p.stderr.readline, b''):
            try:
                line_decoded = line.decode()
            except UnicodeDecodeError:
                # NOTE: Sometimes we get characters back that cannot be decoded(), e.g.
                #
                #   \xa0
                #
                # For example see this in the response from `az group create`:
                #
                # ERROR: Get Token request returned http error: 400 and server 
                # response: {"error":"invalid_grant",# "error_description":"AADSTS700082: 
                # The refresh token has expired due to inactivity.\xa0The token was 
                # issued on 2018-10-25T23:35:11.9832872Z
                #
                # which generates the exception:
                #
                # UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 179: invalid start byte
                #
                print("WARNING: Unable to decode stderr line, printing raw bytes:")
                print(line)
                line_decoded = ""
                pass
            else:

                # azdata emits a single empty line to stderr when doing an hdfs cp, don't
                # print this empty "ERR:" as it confuses.
                #
                if line_decoded == "":
                    continue
                
                print(f"STDERR: {line_decoded}", end='')

                if line_decoded.startswith("An exception has occurred") or line_decoded.startswith("ERROR: An error occurred while executing the following cell"):
                    exit_code_workaround = 1

                # inject HINTs to next TSG/SOP based on output in stderr
                #
                if user_provided_exe_name in error_hints:
                    for error_hint in error_hints[user_provided_exe_name]:
                        if line_decoded.find(error_hint[0]) != -1:
                            display(Markdown(f'HINT: Use [{error_hint[1]}]({error_hint[2]}) to resolve this issue.'))

                # Verify if a transient error, if so automatically retry (recursive)
                #
                if user_provided_exe_name in retry_hints:
                    for retry_hint in retry_hints[user_provided_exe_name]:
                        if line_decoded.find(retry_hint) != -1:
                            if retry_count < MAX_RETRIES:
                                print(f"RETRY: {retry_count} (due to: {retry_hint})")
                                retry_count = retry_count + 1
                                output = run(cmd, return_output=return_output, retry_count=retry_count)

                                if return_output:
                                    if base64_decode:
                                        import base64
                                        return base64.b64decode(output).decode('utf-8')
                                    else:
                                        return output

    elapsed = datetime.datetime.now().replace(microsecond=0) - start_time

    # WORKAROUND: We avoid infinite hang above in the `azdata notebook run` failure case, by inferring success (from stdout output), so
    # don't wait here, if success known above
    #
    if wait: 
        if p.returncode != 0:
            raise SystemExit(f'Shell command:\n\n\t{cmd_display} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(p.returncode)}.\n')
    else:
        if exit_code_workaround !=0 :
            raise SystemExit(f'Shell command:\n\n\t{cmd_display} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(exit_code_workaround)}.\n')

    print(f'\nSUCCESS: {elapsed}s elapsed.\n')

    if return_output:
        if base64_decode:
            import base64
            return base64.b64decode(output).decode('utf-8')
        else:
            return output



# Hints for tool retry (on transient fault), known errors and install guide
#
retry_hints = {'azdata': ['Endpoint sql-server-master does not exist', 'Endpoint livy does not exist', 'Failed to get state for cluster', 'Endpoint webhdfs does not exist', 'Adaptive Server is unavailable or does not exist', 'Error: Address already in use', 'Login timeout expired (0) (SQLDriverConnect)', 'SSPI Provider: No Kerberos credentials available',  ], 'kubectl': ['A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',  ], 'python': [ ], }
error_hints = {'azdata': [['Please run \'azdata login\' to first authenticate', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['The token is expired', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Reason: Unauthorized', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Max retries exceeded with url: /api/v1/bdc/endpoints', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Look at the controller logs for more details', 'TSG027 - Observe cluster deployment', '../diagnose/tsg027-observe-bdc-create.ipynb'], ['provided port is already allocated', 'TSG062 - Get tail of all previous container logs for pods in BDC namespace', '../log-files/tsg062-tail-bdc-previous-container-logs.ipynb'], ['Create cluster failed since the existing namespace', 'SOP061 - Delete a big data cluster', '../install/sop061-delete-bdc.ipynb'], ['Failed to complete kube config setup', 'TSG067 - Failed to complete kube config setup', '../repair/tsg067-failed-to-complete-kube-config-setup.ipynb'], ['Data source name not found and no default driver specified', 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ['Can\'t open lib \'ODBC Driver 17 for SQL Server', 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ['Control plane upgrade failed. Failed to upgrade controller.', 'TSG108 - View the controller upgrade config map', '../diagnose/tsg108-controller-failed-to-upgrade.ipynb'], ['NameError: name \'azdata_login_secret_name\' is not defined', 'SOP013 - Create secret for azdata login (inside cluster)', '../common/sop013-create-secret-for-azdata-login.ipynb'], ['ERROR: No credentials were supplied, or the credentials were unavailable or inaccessible.', 'TSG124 - \'No credentials were supplied\' error from azdata login', '../repair/tsg124-no-credentials-were-supplied.ipynb'], ['Please accept the license terms to use this product through', 'TSG126 - azdata fails with \'accept the license terms to use this product\'', '../repair/tsg126-accept-license-terms.ipynb'],  ], 'kubectl': [['no such host', 'TSG010 - Get configuration contexts', '../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb'], ['No connection could be made because the target machine actively refused it', 'TSG056 - Kubectl fails with No connection could be made because the target machine actively refused it', '../repair/tsg056-kubectl-no-connection-could-be-made.ipynb'],  ], 'python': [['Library not loaded: /usr/local/opt/unixodbc', 'SOP012 - Install unixodbc for Mac', '../install/sop012-brew-install-odbc-for-sql-server.ipynb'], ['WARNING: You are using pip version', 'SOP040 - Upgrade pip in ADS Python sandbox', '../install/sop040-upgrade-pip.ipynb'],  ], }
install_hint = {'azdata': [ 'SOP063 - Install azdata CLI (using package manager)', '../install/sop063-packman-install-azdata.ipynb' ],  'kubectl': [ 'SOP036 - Install kubectl command line interface', '../install/sop036-install-kubectl.ipynb' ],  }


print('Common functions defined successfully.')

### Instantiate Kubernetes client

In [ ]:
# Instantiate the Python Kubernetes client into 'api' variable

import os
from IPython.display import Markdown

try:
    from kubernetes import client, config
    from kubernetes.stream import stream
except ImportError: 

    # Install the Kubernetes module
    import sys
    !{sys.executable} -m pip install kubernetes    
    
    try:
        from kubernetes import client, config
        from kubernetes.stream import stream
    except ImportError:
        display(Markdown(f'HINT: Use [SOP059 - Install Kubernetes Python module](../install/sop059-install-kubernetes-module.ipynb) to resolve this issue.'))
        raise

if "KUBERNETES_SERVICE_PORT" in os.environ and "KUBERNETES_SERVICE_HOST" in os.environ:
    config.load_incluster_config()
else:
    try:
        config.load_kube_config()
    except:
        display(Markdown(f'HINT: Use [TSG118 - Configure Kubernetes config](../repair/tsg118-configure-kube-config.ipynb) to resolve this issue.'))
        raise

api = client.CoreV1Api()

print('Kubernetes client instantiated')

### Helper functions for waiting for the cluster to become healthy

In [ ]:
import threading
import time
import sys
import os
from IPython.display import Markdown

isRunning = True

def all_containers_ready(pod):
    """helper method returns true if all the containers within the given pod are ready

    Arguments:
        pod {v1Pod} -- Metadata retrieved from the api call to.
    """
         
    return all(map(lambda c: c.ready is True, pod.status.container_statuses))


def pod_is_ready(pod):
    """tests that the pod, and all containers are ready

    Arguments:
        pod {v1Pod} -- Metadata retrieved from api call.
    """

    return "job-name" in pod.metadata.labels or (pod.status.phase == "Running" and all_containers_ready(pod))


def waitReady():
    """Waits for all pods, and containers to become ready.
    """
    while isRunning:
        try:
            time.sleep(check_interval)
            pods = get_pods()
            allReady = len(pods.items) >= min_pod_count and all(map(pod_is_ready, pods.items))

            if allReady:
                return True
            else:
                display(Markdown(get_pod_failures(pods)))
                display(Markdown(f"cluster not healthy, rechecking in {check_interval} seconds."))
        except Exception as ex:
            last_error_message = str(ex)
            display(Markdown(last_error_message))
            time.sleep(check_interval)

def get_pod_failures(pods=None):
    """Returns a status message for any pods that are not ready.
    """
    results = ""
    if not pods:
        pods = get_pods()

    for pod in pods.items:
        if "job-name" not in pod.metadata.labels:
            if pod.status and pod.status.container_statuses:
                for container in filter(lambda c: c.ready is False, pod.status.container_statuses):
                    results = results + "Container {0} in Pod {1} is not ready. Reported status: {2} <br/>".format(container.name, pod.metadata.name, container.state)       
            else:
                results = results + "Pod {0} is not ready.  <br/>".format(pod.metadata.name)
    return results


def get_pods():
    """Returns a list of pods by namespace, or all namespaces if no namespace is specified
    """
    pods = None
    if namespace is not None:
        display(Markdown(f'Checking namespace {namespace}'))
        pods = api.list_namespaced_pod(namespace, _request_timeout=30) 
    else:
        display(Markdown('Checking all namespaces'))
        pods = api.list_pod_for_all_namespaces(_request_timeout=30)
    return pods

def wait_for_cluster_healthy():
    isRunning = True
    mt = threading.Thread(target=waitReady)
    mt.start()
    mt.join(timeout=timeout)

    if mt.is_alive():
      raise SystemExit("Timeout waiting for all cluster to be healthy.")
      
    isRunning = False

### Get the Kubernetes namespace for the big data cluster

Get the namespace of the Big Data Cluster use the kubectl command line
interface .

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = run(f'kubectl get namespace --selector=MSSQL_CLUSTER -o jsonpath={{.items[0].metadata.name}}', return_output=True)
    except:
        from IPython.display import Markdown
        print(f"ERROR: Unable to find a Kubernetes namespace with label 'MSSQL_CLUSTER'.  SQL Server Big Data Cluster Kubernetes namespaces contain the label 'MSSQL_CLUSTER'.")
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print(f'The SQL Server Big Data Cluster Kubernetes namespace is: {namespace}')

### Get name of the ‘Running’ `controller` `pod`

In [ ]:
# Place the name  of the 'Running' controller pod in variable `controller`

controller = run(f'kubectl get pod --selector=app=controller -n {namespace} -o jsonpath={{.items[0].metadata.name}} --field-selector=status.phase=Running', return_output=True)

print(f"Controller pod name: {controller}")

### Add Root CA certificate to config map

In [ ]:
import os
import tempfile
from pathlib import Path

cwd = os.getcwd()
path = os.path.join(tempfile.gettempdir(), local_certificate_dir)
os.chdir(path)

bdc_root_ca_name = "cluster-ca-certificate.crt"
bdc_root_ca_common_name = "Cluster Certificate Authority"

run(f'kubectl cp {controller}:/var/run/configmaps/cluster/..data/cluster-ca-certificate.crt {bdc_root_ca_name} -c controller -n {namespace}')

f = open(bdc_root_ca_name, "r")
currentCaCertficates = f.read()
currentCaCertficates = re.sub('(\n)', r'\\r\\n', currentCaCertficates)

f = open(ca_certificate_file_name, "r")
rootCA = f.read()
rootCA = re.sub('(\n)', r'\\r\\n', rootCA)
configmap_certificate_name = Path(ca_certificate_file_name).stem + ".crt"

if configmap_certificate_name == bdc_root_ca_name:
    raise SystemExit(f'Using {bdc_root_ca_name} for CA certificate being uploaded will override existing Big Data Cluster Root CA. Please use different file name.')

run(f'kubectl cp {ca_certificate_file_name} {controller}:/tmp/cer005-{configmap_certificate_name} -c controller -n {namespace}')
cmd = f"openssl x509 -in /tmp/cer005-{configmap_certificate_name} -noout -subject "
subject = run(f'kubectl exec {controller} -c controller -n {namespace} -- bash -c "{cmd}"', return_output=True)
run(f'kubectl exec {controller} -c controller -n {namespace} -- rm /tmp/cer005-{configmap_certificate_name}', return_output=True)

if (subject.find(bdc_root_ca_common_name) != -1):
    raise SystemExit(f'Using {bdc_root_ca_common_name} for subject name of CA certificate being uploaded will override existing Big Data Cluster Root CA. Please use different subject name.')

patchCmd = f'kubectl patch configmap cluster-configmap -n {namespace} --type merge -p \'{{"data":{{"cluster-ca-certificate.crt":"{currentCaCertficates}{rootCA}", "{configmap_certificate_name}":"{rootCA}"}}}}\''

run(patchCmd)

os.chdir(cwd)

### Restart pods to pick up new CA certificate.

In [ ]:
pods = run(f'kubectl get pods -n {namespace} --selector role!=master-pool,role!=controller,app!=clustertest,name!=blockchain-implementation --output=jsonpath={{.items[*].metadata.name}}', return_output=True)

for pod in pods.split(' '):
    run(f'kubectl delete pod {pod} -n {namespace}')
    wait_for_cluster_healthy()

### Get the name of the `master` `pods`

In [ ]:
# Place the name of the master pods in variable `pods`

podNames = run(f'kubectl get pod --selector=app=master -n {namespace} -o jsonpath={{.items[*].metadata.name}}', return_output=True)
pods = podNames.split(" ")

print(f"Master pod names: {pods}")

### Restart Pods

In [ ]:
import threading
import time

if len(pods) == 1:
  # One master pod indicates non-HA environment, just delete it
  run(f'kubectl delete pod {pods[0]} -n {namespace}')
  wait_for_cluster_healthy()
else:
  # HA setup, delete secondaries before primary
  timeout_s = 600
  check_interval_s = 20

  master_primary_svc_ip = run(f'kubectl get service master-p-svc -n {namespace} -o jsonpath={{.spec.clusterIP}}', return_output=True) 
  master_password = run(f'kubectl exec master-0 -c mssql-server -n {namespace} -- cat /var/run/secrets/credentials/pool/mssql-system-password', return_output=True) 

  def get_number_of_unsynchronized_replicas(result):
    cmd = 'select count(*) from sys.dm_hadr_database_replica_states where synchronization_state <> 2'
    res = run(f"kubectl exec controldb-0 -c mssql-server -n {namespace} -- /opt/mssql-tools/bin/sqlcmd -S {master_primary_svc_ip} -U system -P {master_password} -h -1 -q \"SET NOCOUNT ON; {cmd}\" ", return_output=True)
    rows = res.strip().split("\n")

    result[0] = int(rows[0])
    return True

  def get_primary_replica():
    cmd = 'select distinct replica_server_name from sys.dm_hadr_database_replica_states s join sys.availability_replicas r on s.replica_id = r.replica_id where is_primary_replica = 1'
    res = run(f"kubectl exec controldb-0 -c mssql-server -n {namespace} -- /opt/mssql-tools/bin/sqlcmd -S {master_primary_svc_ip} -U system -P {master_password} -h -1 -q \"SET NOCOUNT ON; {cmd}\" ", return_output=True)

    rows = res.strip().split("\n")
    return rows[0]

  def get_secondary_replicas():
    cmd = 'select distinct replica_server_name from sys.dm_hadr_database_replica_states s join sys.availability_replicas r on s.replica_id = r.replica_id where is_primary_replica = 0'
    res = run(f"kubectl exec controldb-0 -c mssql-server -n {namespace} -- /opt/mssql-tools/bin/sqlcmd -S {master_primary_svc_ip} -U system -P {master_password} -h -1 -q \"SET NOCOUNT ON; {cmd}\" ", return_output=True)

    rows = res.strip().split("\n")
    res = []
    for row in rows:
      if (row != "" and "Sqlcmd: Warning" not in row):
        res.append(row.strip())
    return res

  def all_replicas_syncrhonized():
    while True:
      unsynchronized_replicas_cnt = len(pods)
      rows = [None]
      time.sleep(check_interval_s)

      getNumberOfReplicasThread = threading.Thread(target=get_number_of_unsynchronized_replicas, args=(rows,) )
      getNumberOfReplicasThread.start()
      getNumberOfReplicasThread.join(timeout=timeout_s)

      if getNumberOfReplicasThread.is_alive():
        raise SystemExit("Timeout getting the number of unsynchronized replicas.")

      unsynchronized_replicas_cnt = rows[0]
      if (unsynchronized_replicas_cnt == 0):
        return True

  def wait_for_replicas_to_synchronize():
    waitForReplicasToSynchronizeThread = threading.Thread(target=all_replicas_syncrhonized)
    waitForReplicasToSynchronizeThread.start()
    waitForReplicasToSynchronizeThread.join(timeout=timeout_s)

    if waitForReplicasToSynchronizeThread.is_alive():
      raise SystemExit("Timeout waiting for all replicas to be synchronized.")
    
  secondary_replicas = get_secondary_replicas()

  for replica in secondary_replicas:
    wait_for_replicas_to_synchronize()
    run(f'kubectl delete pod {replica} -n {namespace}')

  primary_replica = get_primary_replica() 
  wait_for_replicas_to_synchronize()

  key = "/var/run/secrets/certificates/sqlha/mssql-ha-operator-controller-client/mssql-ha-operator-controller-client-privatekey.pem"
  cert = "/var/run/secrets/certificates/sqlha/mssql-ha-operator-controller-client/mssql-ha-operator-controller-client-certificate.pem"
  content_type_header = "Content-Type: application/json"
  authorization_header = "Authorization: Certificate"
  data = f'{{"TargetReplicaName":"{secondary_replicas[0]}","ForceFailover":"false"}}'
  request_url = f'https://controller-svc:443/internal/api/v1/bdc/services/sql/resources/master/availabilitygroups/containedag/failover'

  manual_failover_api_command = f"curl -sS --key {key} --cert  {cert} -X POST --header '{content_type_header}'  --header '{authorization_header}' --data '{data}' {request_url}"

  operator_pod = run(f'kubectl get pod --selector=app=mssql-operator -n {namespace} -o jsonpath={{.items[0].metadata.name}}', return_output=True)

  run(f'kubectl exec {operator_pod} -c mssql-ha-operator -n {namespace} -- {manual_failover_api_command}')

  wait_for_replicas_to_synchronize()

  run(f'kubectl delete pod {primary_replica} -n {namespace}')
  wait_for_replicas_to_synchronize()

### Get name of the ‘Running’ `controller` `pod`

In [ ]:
# Place the name  of the 'Running' controller pod in variable `controller`

controller = run(f'kubectl get pod --selector=app=controller -n {namespace} -o jsonpath={{.items[0].metadata.name}} --field-selector=status.phase=Running', return_output=True)

print(f"Controller pod name: {controller}")

### Restart controller pod.

In [ ]:
run(f'kubectl delete pod {controller} -n {namespace}')
wait_for_cluster_healthy()

### Wait for controller to be healthy

In [ ]:
import json
import threading
import time

timeout_s = 600
check_interval_s = 10

def get_controller_health(health_map):
    controller_health = run(f"azdata bdc status show --r control", return_output=True)
    controller_health_json = json.loads(controller_health)
    controller_health_status = controller_health_json['healthStatus']

    health_map['controller'] = controller_health_status
    return True

def controller_healthy():
    while True:
        controller_health_status = 'unhealthy'
        health_map = {}
        time.sleep(check_interval_s)

        getControllerHealthThread = threading.Thread(target=get_controller_health, args=(health_map,) )
        getControllerHealthThread.start()
        getControllerHealthThread.join(timeout=timeout_s)

        if getControllerHealthThread.is_alive():
            raise SystemExit("Timeout getting controller health.")

        controller_health_status = health_map['controller'] if 'controller' in health_map else 'unhealthy'
        if (controller_health_status == 'healthy'):
            return True

def wait_for_controller_to_be_healthy():
    waitForControllerToBehealthyThread = threading.Thread(target=controller_healthy)
    waitForControllerToBehealthyThread.start()
    waitForControllerToBehealthyThread.join(timeout=timeout_s)

    if waitForControllerToBehealthyThread.is_alive():
        raise SystemExit("Timeout waiting for controller to be healthy.") 

wait_for_controller_to_be_healthy()

### Update ca certificates on clustertest pod

In [ ]:
pods = api.list_namespaced_pod(namespace, _request_timeout=30)

if any(pod.metadata.name == "clustertest" for pod in pods.items):
    run(f'kubectl exec clustertest -c mssql-test -n {namespace} -- update-ca-certificates')

In [ ]:
print("Notebook execution is complete.")

Related
-------

- [CER001 - Generate a Root CA certificate](../cert-management/cer001-create-root-ca.ipynb)
- [CER010 - Install generated Root CA locally](../cert-management/cer010-install-generated-root-ca-locally.ipynb)
